# Imports

In [2]:
import time
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate
from tensorflow.keras.callbacks import ModelCheckpoint,Callback,LearningRateScheduler , TensorBoard , EarlyStopping
from tensorflow import keras
from tensorflow.keras.optimizers import SGD, RMSprop
from tensorflow.keras import backend as K
import os
import datetime
import numpy as np
import shutil
# from skimage import morphology, feature, segmentation, measure, io, color
# from skimage.filters import sobel
import matplotlib.pyplot as plt
import pdb
import imageio
from scipy import misc
import scipy.ndimage as ndimage
import cv2
from skimage import io, filters, morphology, measure, feature, color
from skimage.segmentation import watershed
from scipy import ndimage as ndi
import pandas as pd
from scipy.ndimage import gaussian_filter, label
from skimage.feature import peak_local_max


# import warnings
# warnings.filterwarnings('ignore')
# from google.colab.patches import cv2_imshow  # Only required in Google Colab
# import warnings
# warnings.filterwarnings("ignore", category=UserWarning, module="imageio")


# Generating DIC like images

In [ ]:
base_address= './Dataset/'

In [ ]:
orig_address=['/train/orig/', '/test/orig/', '/eval/orig/']
dic_address= ['/train/dic/', '/test/dic/', '/eval/dic/']
for i in range(3):

    folder_path = base_address + orig_address[i]
    output_folder_path = base_address+ dic_address[i]

    files = os.listdir(folder_path)
    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for file in files:
        # Create the full file path
        print(os.path.join(folder_path, file))
        if 'ipynb_checkpoints' not in os.path.join(folder_path, file) :

            file_path = os.path.join(folder_path, file)
            # Load the input image
            input_image = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)

            # Apply gradient filter to find edges of the cells
            gradient_x = cv2.Sobel(input_image, cv2.CV_64F, 1, 0, ksize=3)
            gradient_y = cv2.Sobel(input_image, cv2.CV_64F, 0, 1, ksize=3)
            gradient_magnitude = cv2.magnitude(gradient_x, gradient_y)

            # Normalize gradient magnitude to [0, 255]
            gradient_magnitude = cv2.normalize(gradient_magnitude, None, 0, 255, cv2.NORM_MINMAX)

            # Simulate shading
            shading_strength = 0.5 # Adjust this parameter to control shading strength
            shading = np.ones_like(input_image, dtype=np.float64) * 127
            shading += shading_strength * gradient_magnitude

            # Add shading to the input image
            output_image = cv2.add(input_image, shading.astype(np.uint8))

            new_file_name = file.replace('cell.png', '_dic.png')

            output_file_path = os.path.join(output_folder_path, new_file_name)
            print(output_file_path)
            cv2.imwrite(output_file_path, output_image)

# Generating regions images

In [ ]:
def show_maped_centroids(centroids_path , regions_path):
  imList = os.listdir(centroids_path)
  for i in range(len(imList)):
          # read the original image
          if not os.path.exists(output_folder_path):
            os.makedirs(output_folder_path)
          binary_image = cv2.imread( os.path.join(centroids_path, imList[i]) , cv2.IMREAD_GRAYSCALE)
          print(binary_image.shape)
          image_height= binary_image.shape[0]
          image_width=  binary_image.shape[1]
          black_image = np.zeros((image_height, image_width, 1), dtype=np.uint8)

          white_pixel_locations = np.where(binary_image >0)

          for y, x in zip(*white_pixel_locations):
              cv2.circle(black_image, (x, y), 25 , 255, -1)  # Draw a red circle at each white pixel location
              cv2.circle(binary_image, (x, y), 25 , 255, -1)  # Draw a red circle at each white pixel location

          # Show the result image
          # cv2_imshow(black_image)
          # cv2_imshow(binary_image)
          cv2.imwrite( regions_path+ imList[i] + '.png'  , black_image )

          print(str(os.path.join(centroids_path, imList[i])) , "\n\n\n"  )

orig_address=['/train/gt/', '/test/gt/']
dic_address= ['/train/regions/', '/test/regions/']
for i in range(2):
    data = []
    anno = []
    img_labels=[]

    folder_path = base_address + orig_address[i]
    output_folder_path = base_address+ dic_address[i]
    show_maped_centroids(folder_path ,  output_folder_path )